In [1]:
import string
import urllib.request
import csv
import ast
import networkx as nx
import operator
import numpy as np
import numpy.random
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import powerlaw
import nltk 
from wordcloud import WordCloud, ImageColorGenerator 
from PIL import Image 
import urllib.request
from stop_words import get_stop_words
from nltk.corpus import stopwords
from urllib import request
from nltk import word_tokenize
from bs4 import BeautifulSoup

### The functions  in episode data part

In [37]:
def remove_punctuations_from_str(text):
    clean = re.sub(r"""
                [,.;@#?!&$\\\\'\)\("]+  # Accept one or more copies of punctuation
                \ *           # plus zero or more copies of a space,
                """,
                " ",          # and replace it with a single space
                text, flags=re.VERBOSE)
    return clean

def get_single_words_from_str(text_):
    # Remove the envoy symbol
    text = re.sub(r'==.*?==+', '', text_)
    text = text.replace('\n', '')

    List_ = text.split()

    # Remove all punctuation from your list of tokens
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Set everything to lowercase
    text = text.lower()

    # Divide the whole string into words
    tokens = word_tokenize(text)

    # Delete the stopwords in the text
    stop_words = list(get_stop_words('en'))         #Have around 900 stopwords
    nltk_words = list(stopwords.words('english'))   #Have around 150 stopwords
    stop_words.extend(nltk_words)
    tokens_ = []
    for words in tokens:
        if not words in stop_words:
            tokens_.append(words)

    # Remove the words that less than 3 characters
    tokens_A = []
    for i in range(len(tokens_)):
        if len(tokens_[i]) > 3:
            tokens_A.append(tokens_[i])

    return tokens_A

In [18]:
def Get_jsonQuery(episode_name):
    baseurl = "https://rickandmorty.fandom.com/api.php?"
    
    action = "action=query"
    dataformat ="format=json"

    title = "titles="
    title = re.sub(pattern = "$", repl = episode_name, string = title)

    content = "prop=revisions&rvprop=content"

    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    return query

# Get episodes data


In [4]:
episodeName_list = []
seasonNB_list = []
episodeNB_list = []
query_list=[]
href_list = []
seasonNB = 1


# Extract episode from season1 to season5
# Season 6 cannot use this
for i in range(1,6):
    nb=str(i)
    SeasonName =  re.sub(pattern = "($)", repl = nb, string = "Season_")
    line = 'https://rickandmorty.fandom.com/wiki/'
    URL =  re.sub(pattern = "($)", repl = SeasonName, string = line)

    wikiresponse = urllib.request.urlopen(URL)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    soup = BeautifulSoup(wikitext, 'html.parser')

    table = soup.find(id="Episodes").parent.find_next().find_next()
    episode_tr = table.find_all('tr', class_='theme')

    seasonNB = i
    # print(i)
    episodeNB = 1
    for j in range(len(episode_tr)):
        elementEpisode = episode_tr[j].find_all('a')[0]
        episode_name = elementEpisode.string

        episodeName_list.append(episode_name)
        seasonNB_list.append(seasonNB)
        episodeNB_list.append(episodeNB)
    
        episode_href = elementEpisode['href'] #Get href = '/wiki/Title'
        episode_wiki_title = episode_href.split('i/')[1] # Get 'Title'
        # print(episode_wiki_title)
        href_list.append(episode_wiki_title)
        query = Get_jsonQuery(episode_wiki_title) # Get json query
        query_list.append(query)

        episodeNB +=1
    


In [5]:

# Extract episode from season6
URL =  'https://rickandmorty.fandom.com/wiki/Season_6'
wikiresponse = urllib.request.urlopen(URL)
wikidata = wikiresponse.read()
wikitext = wikidata.decode('utf-8')
soup = BeautifulSoup(wikitext, 'html.parser')

table6 = soup.find(id="Episodes").parent.find_next().find_next()
episode_s6_td=table6.find_all('td', style="text-align:left;")
delete=table6.find_all('td', colspan="5",style="text-align:left;")

episode_s6_td = list(episode_s6_td)
Delete = list(delete)
episode_s6_td_ = [x for x in episode_s6_td if x not in Delete]

seasonNB = 6
episodeNB = 1
for j in range(len(episode_s6_td_)):
    elementEpisode = episode_s6_td_[j].find_all('a')[0]

    episode_name = elementEpisode.string

    episodeName_list.append(episode_name)
    seasonNB_list.append(seasonNB)
    episodeNB_list.append(episodeNB)
    
    episode_href = elementEpisode['href'] #Get href = '/wiki/Title'
    episode_wiki_title = episode_href.split('i/')[1] # Get 'Title'
    href_list.append(episode_wiki_title)
    query = Get_jsonQuery(episode_wiki_title) # Get json query
    query_list.append(query)

    episodeNB +=1




In [6]:
title_column = {"episode name":episodeName_list,
                "season":seasonNB_list,
                "episode":episodeNB_list,
                "href title":href_list,
                "query":query_list}
Episode = pd.DataFrame(title_column)


Episode

,episode name,season,episode,href title,query
0,Pilot,1,1,Pilot,https://rickandmorty.fandom.com/api.php?action...
1,Lawnmower Dog,1,2,Lawnmower_Dog,https://rickandmorty.fandom.com/api.php?action...
2,Anatomy Park,1,3,Anatomy_Park_(Episode),https://rickandmorty.fandom.com/api.php?action...
3,M. Night Shaym-Aliens!,1,4,M._Night_Shaym-Aliens!,https://rickandmorty.fandom.com/api.php?action...
4,Meeseeks and Destroy,1,5,Meeseeks_and_Destroy,https://rickandmorty.fandom.com/api.php?action...
...,...,...,...,...,...
56,Juricksic Mort,6,6,Juricksic_Mort,https://rickandmorty.fandom.com/api.php?action...
57,Full Meta Jackrick,6,7,Full_Meta_Jackrick,https://rickandmorty.fandom.com/api.php?action...
58,Analyze Piss,6,8,Analyze_Piss,https://rickandmorty.fandom.com/api.php?action...
59,A Rick in King Mortur's Mort,6,9,A_Rick_in_King_Mortur%27s_Mort,https://rickandmorty.fandom.com/api.php?action...


In [7]:
# Save the content of wikipage (json)
txt_name_list = []
for i in range(len(Episode)):
    query = Episode['query'][i]
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    
    TitleName = Episode['episode name'][i]
    seasonnb = Episode['season'][i]
    E_nb = Episode['episode'][i]
    
    txt_name = "{}_{}_{}.txt".format(seasonnb, E_nb, TitleName)
    txt_name_list.append(txt_name)

    # #Save text as txt
    # f = open(txt_name, "a")
    # f.write(wikitext)
    # f.close()

Episode["txt file name"] = txt_name_list
Episode

,episode name,season,episode,href title,query,txt file name
0,Pilot,1,1,Pilot,https://rickandmorty.fandom.com/api.php?action...,1_1_Pilot.txt
1,Lawnmower Dog,1,2,Lawnmower_Dog,https://rickandmorty.fandom.com/api.php?action...,1_2_Lawnmower Dog.txt
2,Anatomy Park,1,3,Anatomy_Park_(Episode),https://rickandmorty.fandom.com/api.php?action...,1_3_Anatomy Park.txt
3,M. Night Shaym-Aliens!,1,4,M._Night_Shaym-Aliens!,https://rickandmorty.fandom.com/api.php?action...,1_4_M. Night Shaym-Aliens!.txt
4,Meeseeks and Destroy,1,5,Meeseeks_and_Destroy,https://rickandmorty.fandom.com/api.php?action...,1_5_Meeseeks and Destroy.txt
...,...,...,...,...,...,...
56,Juricksic Mort,6,6,Juricksic_Mort,https://rickandmorty.fandom.com/api.php?action...,6_6_Juricksic Mort.txt
57,Full Meta Jackrick,6,7,Full_Meta_Jackrick,https://rickandmorty.fandom.com/api.php?action...,6_7_Full Meta Jackrick.txt
58,Analyze Piss,6,8,Analyze_Piss,https://rickandmorty.fandom.com/api.php?action...,6_8_Analyze Piss.txt
59,A Rick in King Mortur's Mort,6,9,A_Rick_in_King_Mortur%27s_Mort,https://rickandmorty.fandom.com/api.php?action...,6_9_A Rick in King Mortur's Mort.txt


In [31]:
wikitext_list = []
plotText_list=[]
Characters_text_list = []
characters_list = []

# Get  plot text and characters for every episode
for fileName in Episode['txt file name']:
    fileName_ = "data/plot/{}".format(fileName)
    f = open(fileName_, "r")
    wikitext= f.read()
    # wikitext_list.append(wikitext)
    # print(type(wikitext))
    # Get plot text of episodes
    try:
        plottext = re.search(r'==Plot==\\n(.*?)\\n\\n==', wikitext).group(1)
    except:
        try:
            plottext = re.search(r'== Plot ==\\n(.*?)\\n\\n==', wikitext).group(1)
        except:
            plottext = '-'
            print(f"Plot: the Episode {fileName} without plot in the wiki page.")
    
    if len(plottext) < 300:
        plottext = '-'
    
    plottext_ = plottext.replace("\\n", " ")
        
    plotText_list.append(plottext_)



    # Get characters of episodes
    try:
        characters_text = re.search(r'Characters==\\n(.*?)\\n==Deaths', wikitext).group(1)
    except:
        try:
            characters_text = re.search(r'Characters ==\\n(.*?)\\n== Deaths', wikitext).group(1)
        except:
            try:
                characters_text = re.search(r'Characters==\\n(.*?)\\n== Deaths', wikitext).group(1)
            except:
                try:
                    characters_text = re.search(r'characters==\\n(.*?)\\n==Deaths', wikitext).group(1)
                except:
                    try:
                        characters_text = re.search(r'characters==\\n(.*?)\\n== Deaths', wikitext).group(1)
                    except:
                        try:
                            characters_text = re.search(r'characters ==\\n(.*?)\\n== Deaths', wikitext).group(1)
                        except:
                            try:
                                characters_text = re.search(r'characters ==\\n(.*?)\\n==Deaths', wikitext).group(1)
                            except:
                                characters = '-'
                                print(f'Characters: the Episode {fileName} without characters information in the wiki page.')

    exampleText =  characters_text
    # exampleText = "[[character1]] [[character2|blabla]] [[character3 (blabla)]] [[character4#blabla]]"
    pattern1 = "\[\[(.*?)(?:#.*?)?(?:\(.*?)?(?:\|.*?)?\]\]"
    characters = re.findall(pattern1, exampleText)
    if len(characters)<1:
        characters = ['-']
    # print(characters)

    characters_ = []
    for i in range(len(characters)):
        name_new = characters[i].rstrip() # remove white space from the beginning or end of a string
        characters_.append(name_new)

    characters_list.append(characters_)



Plot: the Episode 6_7_Full Meta Jackrick.txt without plot in the wiki page.
Plot: the Episode 6_8_Analyze Piss.txt without plot in the wiki page.
Plot: the Episode 6_9_A Rick in King Mortur's Mort.txt without plot in the wiki page.
Plot: the Episode 6_10_Ricktional Mortpoon's Rickmas Mortcation.txt without plot in the wiki page.


In [10]:
# # Change every name from str to list
# characters_list_new = []
# for i in range(len(characters_list)):
#     test_list = characters_list[i]
#     test_list_new = []
#     for j in range(len(test_list)):
#         c = test_list[j].translate(str.maketrans('', '', string.punctuation))
#         # Remove the punctuations in case of errors with saving and loading
#         # a = c.split("delimiter")
#         test_list_new.append(c)
#     characters_list_new.append(test_list_new)


# Save plot and character names into Eposide
Episode['characters of Episodes'] = characters_list
Episode['plot text'] = plotText_list



### Extract the script for every episode

In [2]:
# extract the script for every episode
Episode = pd.read_csv('episode.csv') #

Transcriptbox_query_list = []
for fileName in Episode['href title']:
    baseurl = "https://rickandmorty.fandom.com/api.php?"

    action = "action=query"
    dataformat ="format=json"

    title = "titles="
    title = repl = "titles={}/Transcript".format(fileName)

    content = "prop=revisions&rvprop=content"

    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    Transcriptbox_query_list.append(query)

In [20]:
script_name_list=[]
Transcript_list = []
for i in range(len(Transcriptbox_query_list)):
    query = Transcriptbox_query_list[i]
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext_ = wikidata.decode('utf-8')

    #handle the json text get clean transcript strings
    wikitext_ = str(wikitext_)
    s = wikitext_
    # s = 'Part 1. Part 2. Part 3 then more text'
    a, b = s.find('==Transcript=='), s.find('==Site navigation==')
    wikitext_1 = s[a+16:b]
    wikitext_2 = wikitext_1.replace("\\n", " ")
    wikitext_3 = wikitext_2.replace("\'", " ")
    wikitext_4 = wikitext_3.replace("\\u", " ")

    # Transcript_list.append(wikitext_4)
    TitleName = Episode['episode name'][i]
    seasonnb = Episode['season'][i]
    E_nb = Episode['episode'][i]
    script_name = "script_{}_{}_{}_.txt".format(seasonnb, E_nb, TitleName)
    script_name_list.append(script_name)
    # #Save
    # f = open(script_name , "a")
    # f.write(wikitext_4)
    # f.close()


In [11]:
for fileName in Episode['txt file name']:
    fileName_ = "data/{}".format(fileName)
    f = open(fileName_, "r")
    wikitext= f.read()

In [23]:
scriptList = []
for scriptName in script_name_list:
    scriptName_ = "data/script/{}".format(scriptName)
    f = open(scriptName_, "r")
    scripttext= f.read()
    scriptList.append(scripttext)

In [30]:
Episode['scriptList']=scriptList
Episode['plot text'][60]

'-'

In [32]:
# save episode data as csx
Episode.to_csv('episode.csv', index=False)

# Load episodes data from csv

PS:Season 6 has been broadcast 6 episodes, the last 4 episodes have wiki pages but there are no plot and characters there.

In [35]:
Episode_load = pd.read_csv('episode.csv')
Episode_load

,episode name,season,episode,href title,query,txt file name,characters of Episodes,plot text,scriptList
0,Pilot,1,1,Pilot,https://rickandmorty.fandom.com/api.php?action...,1_1_Pilot.txt,"['Rick Sanchez', 'Morty Smith', 'Jerry Smith',...","In the middle of the night, an obviously drunk...",<poem> [Episode begins in Morty 2019s room whe...
1,Lawnmower Dog,1,2,Lawnmower_Dog,https://rickandmorty.fandom.com/api.php?action...,1_2_Lawnmower Dog.txt,"['Morty', 'Rick', 'Jerry Smith', 'Cronenberged...",[[Jerry Smith (C-137)|Jerry]] complains that t...,<poem> ([[Jerry]] and [[Summer]] are in the ...
2,Anatomy Park,1,3,Anatomy_Park_(episode),https://rickandmorty.fandom.com/api.php?action...,1_3_Anatomy Park.txt,"['Rick Sanchez', 'Morty Smith', 'Beth Smith', ...","It's Christmas, and [[Jerry Smith (C-137)|Jerr...","[Open Ext. Smith residence] (Outside, there i..."
3,M. Night Shaym-Aliens!,1,4,M._Night_Shaym-Aliens!,https://rickandmorty.fandom.com/api.php?action...,1_4_M. Night Shaym-Aliens!.txt,"['Rick Sanchez', 'Jerry Smith', 'Prince Nebulo...",The episode opens with [[Rick Sanchez|Rick]] d...,[Open Int. Rick s garage.] ([[Rick Sanchez|Ri...
4,Meeseeks and Destroy,1,5,Meeseeks_and_Destroy,https://rickandmorty.fandom.com/api.php?action...,1_5_Meeseeks and Destroy.txt,"['Rick', 'Morty', 'Mr. Meeseeks', 'Jerry Smith...",[[Image:Meeseeks and Destroy 6.png|thumb|200px...,"[Open Ext. Damaged space station, likely in an..."
...,...,...,...,...,...,...,...,...,...
56,Juricksic Mort,6,6,Juricksic_Mort,https://rickandmorty.fandom.com/api.php?action...,6_6_Juricksic Mort.txt,"['Rick Sanchez', 'Morty Smith', 'Beth Smith', ...",-,""":"""",""query"":{""normalized"":[{""from"":""Juricksic..."
57,Full Meta Jackrick,6,7,Full_Meta_Jackrick,https://rickandmorty.fandom.com/api.php?action...,6_7_Full Meta Jackrick.txt,"['Rick Sanchez', 'Morty Smith', 'Beth Smith', ...",-,-
58,Analyze Piss,6,8,Analyze_Piss,https://rickandmorty.fandom.com/api.php?action...,6_8_Analyze Piss.txt,['-'],-,-
59,A Rick in King Mortur's Mort,6,9,A_Rick_in_King_Mortur%27s_Mort,https://rickandmorty.fandom.com/api.php?action...,6_9_A Rick in King Mortur's Mort.txt,['-'],-,-


In [38]:
# The exapmle getting plot text and tokens
test_plot = Episode_load['plot text'][0]
# print(test_plot)
tokens = get_single_words_from_str(remove_punctuations_from_str(test_plot))
tokens 

['middle',
 'night',
 'obviously',
 'drunk',
 'rick',
 'bursts',
 'morty',
 'room',
 'tells',
 'surprise',
 'show',
 'driving',
 'newlybuilt',
 'space',
 'cruiserflying',
 'vehicle',
 'rick',
 'explains',
 'made',
 'neutrino',
 'bomb',
 'decided',
 'earth',
 'needs',
 'fresh',
 'start',
 'wants',
 'destroy',
 'humanity',
 'planning',
 'start',
 'morty',
 'jessica',
 'adam',
 'morty',
 'tries',
 'take',
 'control',
 'situation',
 'taking',
 'wheel',
 'vehicle',
 'rick',
 'eventually',
 'decides',
 'land',
 'vehicle',
 'says',
 'test',
 'morty',
 'assertive',
 'passes',
 'bomb',
 'announces',
 'armed',
 'presumably',
 'activates',
 'next',
 'morning',
 'morty',
 'falls',
 'asleep',
 'breakfast',
 'mother',
 'beth',
 'smith',
 'cronenberged',
 'dimension',
 'beth',
 'concerned',
 'morty',
 'older',
 'sister',
 'summer',
 'smith',
 'cronenberged',
 'dimension',
 'summer',
 'explains',
 'grandpa',
 'rick',
 'causes',
 'argument',
 'jerry',
 'smith',
 'cronenberged',
 'dimension',
 'jerry',


In [39]:
# The example getting characters list of the episode
test_characters = Episode_load['characters of Episodes'][1]
test_characters_ = ast.literal_eval(test_characters)
test_characters_

['Morty',
 'Rick',
 'Jerry Smith',
 'Cronenberged dimension',
 'Snuffles',
 'Scary Terry',
 'Mr. Goldenfold',
 'Beth Smith',
 'Summer Smith',
 'Centaur',
 'Melissa',
 'Scary Brandon',
 'Mrs. Pancakes',
 'Creepy Little Girl',
 'Scary Olderson',
 'Scary Glenn',
 'Bill']

In [42]:
# The exapmle getting script
test_script = Episode_load['scriptList'][45]